In [ ]:
#!/usr/bin/python

import socket
import sys
import re
import time
import psycopg2

from datetime import datetime, timezone
from AtlasI2C import AtlasI2C
    
def detectSensors():
    device_list = []
    dev = AtlasI2C()
    for address in dev.list_i2c_devices():
        dev.set_i2c_address(address)
        if 11 <= address <= 127:
            moduletype = "ERR"
            while moduletype == "ERR":
                response = dev.query("I")
                if ("HUM" in response):
                    moduletype = "HUM"
                elif ("CO2" in response):
                    moduletype = "CO2"
                elif ("PRS" in response):
                    moduletype = "PRS"
                else:
                    error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)
            name = "UNKNOWN"
            while (name == "UNKNOWN") or (bool == False):
                bool = False 
                response = dev.query("name,?")
                if ("NAME," in response):
                    name = response.split(",")[1]
                    name = name[:9]
                    r = re.compile('.{3}-.{2}-.{2}')
                    if r.match(name):
                        bool = True
                else:
                    error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                    sys.stderr.write(error_message)

            device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

    for device in device_list:
        print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    return device_list

def setProperties(device_list):
    for device in device_list:
        if (device.moduletype == "HUM"):
            device.write("O,T,1")
            time.sleep(1)
            device.write("O,Dew,1")
            time.sleep(1)
        if (device.moduletype == "CO2"):
            device.write("O,t,1")
            time.sleep(1)

def writeHeader(output_file, device_list):
    csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
    output_file.write(csv_header+"\n")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        device_output = device.read()
        device_output.encode('ascii')
        device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
    except:
        print("Reading I2C device failed: "+device.name)
        return reading, str(device_output)

    if (device.moduletype == "HUM"):
        try:
            data = device_output.split(': ')[1]
            hum = float(data.split(',')[0])
            temp = float(data.split(',')[1])
            dew = float(data.split(',')[3])
        except:
            hum = 0.0
            temp = 0.0
            dew = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device:" 
        reading += str(hum) + ", "
        reading += str(temp) + ", "
        reading += str(dew)
    if (device.moduletype == "CO2"):
        try:
            data = device_output.split(': ')[1]
            co2 = float(data.split(',')[0])
            temp = float(data.split(',')[1])
        except:
            co2 = 0.0
            temp = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(co2) + ", "
        reading += str(temp)
    if (device.moduletype == "PRS"):
        try:
            data = device_output.split(': ')[1]
            prs = data
        except:
            prs = 0.0
            error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
            sys.stderr.write(error_message)
            errMsg = "Unexpected reading on device"
        reading += str(prs)

    return reading, str(device_output), errMsg

def main():
    host = "unemit-research.postgres.database.azure.com"
    dbname = "sensor_data"
    user = "clerk"
    password = "A*shht6Qy5&GQl29"
    sslmode = "require"
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string) 
    print("Database connection established")
    cursor = conn.cursor()
    
    device_list = detectSensors()    
    if (not device_list):
        sys.stderr.write("No sensors detected. Quitting...\n")
        sys.exit(1)

    setProperties(device_list)

    output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
    header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
        error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    with open(output_filename, "w", buffering=1) as output_file:
        writeHeader(output_file, device_list)
        while True:
            requestReading(device_list)
            time.sleep(1)
            for device in device_list:
                currentReading, probeString, errorMessage = pollDevice(device)
                data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                date_time = datetime.now()
                data += ", "+str(socket.gethostname()) + ", " + device.name
                if (device.moduletype == "HUM"):
                    data1 = data + ", humidity, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", temperature," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    data3 = data + ", dew point," + currentReading.split(',')[2]
                    output_file.write(str(data3)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "humidity", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "temperature", currentReading.split(',')[1], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "dew point", currentReading.split(',')[2], \
                        errorMessage, probeString))                    
                                            
                if (device.moduletype == "CO2"):
                    data1 = data + ", CO2, " + currentReading.split(',')[0]
                    output_file.write(str(data1)+"\n")
                    data2 = data + ", sensor temp," + currentReading.split(',')[1]
                    output_file.write(str(data2)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "CO2", currentReading.split(',')[0], \
                        errorMessage, probeString))
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "sensor temp", currentReading.split(',')[1], \
                        errorMessage, probeString))  
                                         
                if (device.moduletype == "PRS"):
                    data1 = data + ", pressure, " + currentReading
                    output_file.write(str(data1)+"\n")
                    
                    cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                        device.name, "pressure", currentReading, \
                        errorMessage, probeString))                 
                conn.commit()
            time.sleep(10)

if __name__ == '__main__':
    main()

In [ ]:
# CODE WITH PRINTS 

In [1]:
try:
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
    import traceback
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

    
def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        return "", "", "Unexpected error"

#~~~~~~CONSTANTS~~~~~~
# CONSTANTS
N = 5

co2_readings = deque(maxlen=N)
hum_readings = deque(maxlen=N)
temp_readingss = deque(maxlen=N)
temp_readings = deque(maxlen=N)

co2_queue = queue.Queue()
hum_queue = queue.Queue()
temp_queue = queue.Queue()

def main():

    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:

                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        # get HUM data
                        hum_level = float(currentReading.split(',')[0])
                        if hum_level > 0:
                            hum_readings.append(hum_level)
                            hum_queue.put(("Humidity", hum_level))
                        avg_hum = sum(hum_readings) / len(hum_readings) if hum_readings else 0
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")

                        # get TEMP data
                        temp_level = float(currentReading.split(',')[1])
                        if temp_level > 0 :
                            temp_readings.append(temp_level)
                            temp_queue.put(("temperature", temp_level))
                            #print(f"temp level for {device.name}: {temp_level}")
                            print(f"temp readings: {temp_readings}")
                        #avg_temp = sum(temp_readings) / len(temp_readings) if temp_readings else 0
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        # get CO2 data
                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_readings.append(co2_level)
                            co2_queue.put(("CO2", co2_level))
                        #avg_co2 = sum(co2_readings) / len(co2_readings) if co2_readings else 0

                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))

                    conn.commit()

                loop_counter += 1
                print(loop_counter)
                time.sleep(10)

    except Exception as e:
        logging.error(f"Unexpected error in main loop: {e}")
    finally:
        if conn:
            conn.close()
        GPIO.cleanup()


if __name__ == '__main__':
    main()

NameError: name 'logging' is not defined